In [ ]:
###### GLOBAL ###########

NLTK_ERROR_HAPPENING = False

MAKE_DATA = False
ROWS = 70000 # LIMIT = 106062
DF1, DF2, DF3 = False, True, False
ALL_ROWS = False

PREPROCESS = False

MODEL_TYPE = 'SGD'

NEW_MODELS = False

PARAMETER_SEARCH = False

TRAIN = False

PREDICT_TEST = False

###### GLOBAL ###########

In [2]:
if NLTK_ERROR_HAPPENING == True
  import nltk
  nltk.download('punkt')
  nltk.download('stopwords')
  nltk.download('wordnet')

import pandas as pd
import pickle

from data.data import Data
from scripts.Preprocessing_full import *
from model.model import *


if MAKE_DATA == True:
  data_raw = Data().get_all_data()

  # 8670 ROWS INPUT #
  if DF1:
    df1 = data_raw['twitter_MBTI']
    df1 = df1[['label', 'text']]
    df1.rename(columns={'label': 'type'}, inplace=True)
    if ALL_ROWS == False:
      df1 = df1.sample(n = ROWS, ignore_index = True)


  # 106062 ROWS INPUT #
  if DF2:
    df2 = data_raw['MBTI 500']
    df2 = df2[['type', 'posts']]
    df2.rename(columns={'posts': 'text'}, inplace=True)
    if ALL_ROWS == False:
      df2 = df2.sample(n = ROWS, ignore_index = True) 


  # 7811 ROWS INPUT #
  if DF3:
    df3 = data_raw['mbti_1']
    df3.rename(columns={'posts': 'text'}, inplace=True)
    if ALL_ROWS == False:
      df3 = df3.sample(n = ROWS, ignore_index = True)

  #####################################################



  ## Combined all data for PP if desired
  data_combined = pd.concat([df1, df2, df3], ignore_index=True)

else:
  print("Data ready")


In [3]:
from scripts.Preprocessing_full import training_preprocessing, training_oversampling, training_balancing, training_vectorize

if PREPROCESS:
  df_pp = training_preprocessing(data_combined)
  df_pp_os = training_oversampling(df_pp)
  df_pp_os_bal = training_balancing(df_pp_os)
  df_list = training_vectorize(df_pp_os_bal)

  df_list[0].dropna(inplace = True)
  df_list[1].dropna(inplace = True)
  df_list[2].dropna(inplace = True)
  df_list[3].dropna(inplace = True)

  with open(f'dflist_pp_{ROWS}.pkl', 'wb') as file:
    pickle.dump(df_list, file)

else:
  with open(f'dflist_pp_{ROWS}.pkl', 'rb') as file:
    df_list = pickle.load(file)
  print("df_list loaded")

<!--  -->

In [94]:
if NEW_MODELS:
  model_list = initialize_models(new_models = True,
                               params = None,
                               model_type = MODEL_TYPE)
  print("model_list is NEW")
else:
  model_list = load_models_pkl()
  print("model_list is OLD")


[SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000),
 SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000),
 SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000),
 SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000)]

In [ ]:
if PARAMETER_SEARCH:
  param_list = grid_search_all_models(df_list, 
                          model_type = MODEL_TYPE)
  with open(f'param_list_sgd.pkl', 'wb') as file:
    pickle.dump(param_list, file)
  
  model_list = initialize_models(params = param_list,
                               model_type=MODEL_TYPE)
  print("model_list is optimized on new params")
  
else:
  with open(f'param_list_sgd.pkl', 'rb') as file:
    param_list = pickle.load(file)

  model_list = initialize_models(params = param_list,
                               model_type=MODEL_TYPE) 
  
  print("model_list is optimized on old params")

In [43]:
if TRAIN:
  history = train_model(df_list)
else:
  print("Not Training")

F1-score: 72.395%
Model: SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000, penalty='l1')
MBTI Type: EI
F1-score: 77.11%
Model: SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000, penalty='l1')
MBTI Type: NS
F1-score: 79.228%
Model: SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000,
              penalty='elasticnet')
MBTI Type: FT
F1-score: 65.688%
Model: SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000,
              penalty='elasticnet')
MBTI Type: JP


In [90]:
from scripts.Preprocessing_full import prediction_preprocessing, prediction_vectorize
if not MAKE_DATA:
  data_raw = Data().get_all_data()
  df2 = data_raw['MBTI 500']
  df2 = df2[['type', 'posts']]
  df2.rename(columns={'posts': 'text'}, inplace=True)

if PREDICT_TEST:
  j = 0
  for i in range(100):
    df = df2.sample(n = 10, ignore_index = True)
    estimation = df['type'].iloc[:1].tolist()[0]
    df_pred = df.iloc[:1].drop(columns =['type'])

    df_pred_pp = prediction_preprocessing(df_pred)
    df_pred_list = prediction_vectorize(df_pred_pp)
    pred = predict_model(texts = df_pred_list, verbose = False)
    # print(f"Prediction = {pred['type_prediction']} {pred['type_score']}")
    # print(f"Actual = {estimation}")
    if pred['type_prediction'] == estimation:
      j += 1
  print(f"{j} correct out of {i}")

37 correct out of 99


# DF Comparison (n=7000)
### DF1 ------------------------------------------
F1-score: 52.669999999999995%
MBTI Type: EI

F1-score: 53.102000000000004%
MBTI Type: NS

F1-score: 49.124%
MBTI Type: TF

F1-score: 57.786% 
MBTI Type: PJ

### DF2 ------------------------------------------
F1-score: 60.718%
MBTI Type: EI
 
F1-score: 60.75000000000001%
MBTI Type: NS 

F1-score: 78.408%
MBTI Type: TF

F1-score: 54.04600000000001%
MBTI Type: PJ


### DF 3 -------------------------------------


F1-score: 58.536%
MBTI Type: IE

F1-score: 58.318999999999996%
MBTI Type: NS

F1-score: 66.681%
MBTI Type: TF

F1-score: 57.265%
MBTI Type: JP

